## MASTER DATA SCIENCE: NUCLIO
## PROFESOR: ALBERTO VACAS
## PROJECT TRENMAX

### IMPORTAMOS LIBRERÍAS Y LEEMOS EL FICHERO

In [ ]:
# IMPORTAMOS LIBRERÍAS

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

In [ ]:
# LEEMOS EL FICHERO DE RENFE

df_renfe = pd.read_csv("renfe.csv", sep=",", encoding='latin-1')
df_renfe.head()

### PARTE 1

#### PREGUNTA 1

¿Qué variables tienen nulos?

In [ ]:
df_renfe.isnull().sum()

In [ ]:
df_renfe.isnull().any()

#### PREGUNTA 2

Convertimos las fechas a formato datetime.

In [ ]:
df_renfe.info()

In [ ]:
for i in ["FECHA_CONSULTA","FECHA_INICIO","FECHA_FIN"]:
    df_renfe[i] = pd.to_datetime(df_renfe[i])

In [ ]:
df_renfe.info()

#### PREGUNTA 3

Eliminamos duplicados.

In [ ]:
df_renfe[df_renfe.duplicated(keep='first')]

In [ ]:
df_renfe[df_renfe.duplicated()].shape

In [ ]:
df_renfe_sin_duplicados = df_renfe.drop_duplicates().copy()

In [ ]:
df_renfe_sin_duplicados.head()

In [ ]:
df_renfe.shape

In [ ]:
df_renfe_sin_duplicados.shape

In [ ]:
df_renfe_sin_duplicados.tail()

In [ ]:
df_renfe_sin_duplicados.reset_index(drop=True, inplace=True)

In [ ]:
df_renfe_sin_duplicados.tail()

In [ ]:
df_renfe_sin_duplicados[df_renfe_sin_duplicados.duplicated()].shape

In [ ]:
df_renfe_sin_duplicados[df_renfe_sin_duplicados.duplicated()]

#### PREGUNTA 4

Creamos las variables TIEMPO_VIAJE (variable que indica en minutos la duración del viaje) y DIF_INI_BUS (variable que indica en minutos la diferencia entre la fecha de inicio de viaje y la fecha de búsqueda).

In [ ]:
df_renfe_sin_duplicados["TIEMPO_VIAJE"] = (df_renfe_sin_duplicados["FECHA_FIN"]-df_renfe_sin_duplicados["FECHA_INICIO"])/np.timedelta64(1,'m')

In [ ]:
df_renfe_sin_duplicados["DIF_INI_BUS"] = (df_renfe_sin_duplicados["FECHA_INICIO"]-df_renfe_sin_duplicados["FECHA_CONSULTA"])/np.timedelta64(1,'m')

In [ ]:
df_renfe_sin_duplicados.head()

#### PREGUNTA 5

Este código separa las fechas en hora, día (nombre), día (número) y mes.

In [ ]:
for col in ['FECHA_CONSULTA', 'FECHA_INICIO', 'FECHA_FIN']:

    df_renfe_sin_duplicados[col + '_HORA'] = df_renfe_sin_duplicados[col].dt.hour
    df_renfe_sin_duplicados[col + '_NOMBREDIA'] = df_renfe_sin_duplicados[col].dt.day_name()
    df_renfe_sin_duplicados[col + '_DIA'] = df_renfe_sin_duplicados[col].dt.day
    df_renfe_sin_duplicados[col + '_MES'] = df_renfe_sin_duplicados[col].dt.month

    del(df_renfe_sin_duplicados[col])

df_renfe_sin_duplicados.head()

#### PREGUNTA 6

Calculamos el porcentaje de nulos.

In [ ]:
df_renfe_sin_duplicados["PRECIO"].isnull().sum()

In [ ]:
df_renfe_sin_duplicados.shape[0]

In [ ]:
(df_renfe_sin_duplicados["PRECIO"].isnull().sum()/df_renfe_sin_duplicados.shape[0])*100

In [ ]:
df_renfe_sin_duplicados["PRECIO"].value_counts(dropna=False,normalize=True)*100

#### PREGUNTA 7

Analizamos y limpiamos los nulos.

In [ ]:
df_renfe_sin_duplicados.isnull().sum()

In [ ]:
df_renfe_sin_duplicados.info()

In [ ]:
df_renfe_sin_duplicados.hist("PRECIO")

In [ ]:
df_renfe_sin_duplicados["TIPO_TREN"].value_counts()

In [ ]:
df_renfe_sin_duplicados["TIPO_TARIFA"].value_counts()

In [ ]:
df_renfe_sin_duplicados["CLASE"].value_counts()

Analizamos variable target

In [ ]:
df_renfe_sin_duplicados_nulos = df_renfe_sin_duplicados.copy()

In [ ]:
df_renfe_sin_duplicados_nulos.dropna(subset=["PRECIO"], inplace=True) ##Eliminamos los nulos del target

Analizamos resto variable con nulos

In [ ]:
df_renfe_sin_duplicados_nulos[(df_renfe_sin_duplicados_nulos['TIPO_TREN'].isnull()) &
                              (df_renfe_sin_duplicados_nulos['TIPO_TARIFA'].isnull()) &
                              (df_renfe_sin_duplicados_nulos['CLASE'].isnull())]

In [ ]:
df_renfe_sin_duplicados_nulos.shape

In [ ]:
df_renfe_sin_duplicados_nulos.dropna(subset=["TIPO_TREN","TIPO_TARIFA","CLASE"], how='all', inplace=True) ##Eliminamos los que tengan las 3 columnas a nulo

In [ ]:
df_renfe_sin_duplicados_nulos.shape

Variable Clase

In [ ]:
COLUMNS_1 = ['CIUDAD_ORIGEN','CIUDAD_DESTINO',"TIPO_TREN","TIPO_TARIFA"]
df_nulls_1 = df_renfe_sin_duplicados_nulos.groupby(COLUMNS_1)['CLASE'].agg(lambda x: pd.Series.mode(x).iloc[0] if not pd.Series.mode(x).empty else np.nan).to_frame().reset_index()
df_nulls_1 = df_nulls_1.dropna(subset=['CLASE'])
df_nulls_1.rename(columns={"CLASE":"CLASE_1"},inplace=True)

COLUMNS_2 = ['CIUDAD_ORIGEN','CIUDAD_DESTINO',"TIPO_TREN"]
df_nulls_2 = df_renfe_sin_duplicados_nulos.groupby(COLUMNS_2)['CLASE'].agg(lambda x: pd.Series.mode(x).iloc[0] if not pd.Series.mode(x).empty else np.nan).to_frame().reset_index()
df_nulls_2 = df_nulls_2.dropna(subset=['CLASE'])
df_nulls_2.rename(columns={"CLASE":"CLASE_2"},inplace=True)

COLUMNS_3 = ['CIUDAD_ORIGEN','CIUDAD_DESTINO',"TIPO_TARIFA"]
df_nulls_3 = df_renfe_sin_duplicados_nulos.groupby(COLUMNS_3)['CLASE'].agg(lambda x: pd.Series.mode(x).iloc[0] if not pd.Series.mode(x).empty else np.nan).to_frame().reset_index()
df_nulls_3 = df_nulls_3.dropna(subset=['CLASE'])
df_nulls_3.rename(columns={"CLASE":"CLASE_3"},inplace=True)

In [ ]:
df_renfe_sin_duplicados_nulos = df_renfe_sin_duplicados_nulos.merge(df_nulls_1,how='left',left_on=COLUMNS_1,right_on=COLUMNS_1)
df_renfe_sin_duplicados_nulos = df_renfe_sin_duplicados_nulos.merge(df_nulls_2,how='left',left_on=COLUMNS_2,right_on=COLUMNS_2)
df_renfe_sin_duplicados_nulos = df_renfe_sin_duplicados_nulos.merge(df_nulls_3,how='left',left_on=COLUMNS_3,right_on=COLUMNS_3)

df_renfe_sin_duplicados_nulos['CLASE'] = df_renfe_sin_duplicados_nulos[['CLASE','CLASE_1','CLASE_2','CLASE_3']].bfill(axis=1).bfill(axis=1).iloc[:, 0]

df_renfe_sin_duplicados_nulos.drop(columns=['CLASE_1','CLASE_2','CLASE_3'],inplace=True)

Variable Tipo tarifa

In [ ]:
COLUMNS_1 = ['CIUDAD_ORIGEN','CIUDAD_DESTINO',"TIPO_TREN","CLASE"]
df_nulls_1 = df_renfe_sin_duplicados_nulos.groupby(COLUMNS_1)['TIPO_TARIFA'].agg(lambda x: pd.Series.mode(x).iloc[0] if not pd.Series.mode(x).empty else np.nan).to_frame().reset_index()
df_nulls_1 = df_nulls_1.dropna(subset=['TIPO_TARIFA'])
df_nulls_1.rename(columns={"TIPO_TARIFA":"TIPO_TARIFA_1"},inplace=True)

COLUMNS_2 = ['CIUDAD_ORIGEN','CIUDAD_DESTINO',"TIPO_TREN"]
df_nulls_2 = df_renfe_sin_duplicados_nulos.groupby(COLUMNS_2)['TIPO_TARIFA'].agg(lambda x: pd.Series.mode(x).iloc[0] if not pd.Series.mode(x).empty else np.nan).to_frame().reset_index()
df_nulls_2 = df_nulls_2.dropna(subset=['TIPO_TARIFA'])
df_nulls_2.rename(columns={"TIPO_TARIFA":"TIPO_TARIFA_2"},inplace=True)

COLUMNS_3 = ['CIUDAD_ORIGEN','CIUDAD_DESTINO',"CLASE"]
df_nulls_3 = df_renfe_sin_duplicados_nulos.groupby(COLUMNS_3)['TIPO_TARIFA'].agg(lambda x: pd.Series.mode(x).iloc[0] if not pd.Series.mode(x).empty else np.nan).to_frame().reset_index()
df_nulls_3 = df_nulls_3.dropna(subset=['TIPO_TARIFA'])
df_nulls_3.rename(columns={"TIPO_TARIFA":"TIPO_TARIFA_3"},inplace=True)

In [ ]:
df_renfe_sin_duplicados_nulos = df_renfe_sin_duplicados_nulos.merge(df_nulls_1,how='left',left_on=COLUMNS_1,right_on=COLUMNS_1)
df_renfe_sin_duplicados_nulos = df_renfe_sin_duplicados_nulos.merge(df_nulls_2,how='left',left_on=COLUMNS_2,right_on=COLUMNS_2)
df_renfe_sin_duplicados_nulos = df_renfe_sin_duplicados_nulos.merge(df_nulls_3,how='left',left_on=COLUMNS_3,right_on=COLUMNS_3)

df_renfe_sin_duplicados_nulos['TIPO_TARIFA'] = df_renfe_sin_duplicados_nulos[['TIPO_TARIFA','TIPO_TARIFA_1','TIPO_TARIFA_2','TIPO_TARIFA_3']].bfill(axis=1).bfill(axis=1).iloc[:, 0]

df_renfe_sin_duplicados_nulos.drop(columns=['TIPO_TARIFA_1','TIPO_TARIFA_2','TIPO_TARIFA_3'],inplace=True)

Variable Tipo tren

In [ ]:
COLUMNS_1 = ['CIUDAD_ORIGEN','CIUDAD_DESTINO',"TIPO_TARIFA","CLASE"]
df_nulls_1 = df_renfe_sin_duplicados_nulos.groupby(COLUMNS_1)['TIPO_TREN'].agg(lambda x: pd.Series.mode(x).iloc[0] if not pd.Series.mode(x).empty else np.nan).reset_index()
df_nulls_1 = df_nulls_1.dropna(subset=['TIPO_TREN'])
df_nulls_1.rename(columns={"TIPO_TREN":"TIPO_TREN_1"},inplace=True)

COLUMNS_2 = ['CIUDAD_ORIGEN','CIUDAD_DESTINO',"TIPO_TARIFA"]
df_nulls_2 = df_renfe_sin_duplicados_nulos.groupby(COLUMNS_2)['TIPO_TREN'].agg(lambda x: pd.Series.mode(x).iloc[0] if not pd.Series.mode(x).empty else np.nan).to_frame().reset_index()
df_nulls_2 = df_nulls_2.dropna(subset=['TIPO_TREN'])
df_nulls_2.rename(columns={"TIPO_TREN":"TIPO_TREN_2"},inplace=True)

COLUMNS_3 = ['CIUDAD_ORIGEN','CIUDAD_DESTINO',"CLASE"]
df_nulls_3 = df_renfe_sin_duplicados_nulos.groupby(COLUMNS_3)['TIPO_TREN'].agg(lambda x: pd.Series.mode(x).iloc[0] if not pd.Series.mode(x).empty else np.nan).to_frame().reset_index()
df_nulls_3 = df_nulls_3.dropna(subset=['TIPO_TREN'])
df_nulls_3.rename(columns={"TIPO_TREN":"TIPO_TREN_3"},inplace=True)

In [ ]:
df_renfe_sin_duplicados_nulos = df_renfe_sin_duplicados_nulos.merge(df_nulls_1,how='left',left_on=COLUMNS_1,right_on=COLUMNS_1)
df_renfe_sin_duplicados_nulos = df_renfe_sin_duplicados_nulos.merge(df_nulls_2,how='left',left_on=COLUMNS_2,right_on=COLUMNS_2)
df_renfe_sin_duplicados_nulos = df_renfe_sin_duplicados_nulos.merge(df_nulls_3,how='left',left_on=COLUMNS_3,right_on=COLUMNS_3)

df_renfe_sin_duplicados_nulos['TIPO_TREN'] = df_renfe_sin_duplicados_nulos[['TIPO_TREN','TIPO_TREN_1','TIPO_TREN_2','TIPO_TREN_3']].bfill(axis=1).bfill(axis=1).iloc[:, 0]

df_renfe_sin_duplicados_nulos.drop(columns=['TIPO_TREN_1','TIPO_TREN_2','TIPO_TREN_3'],inplace=True)

In [ ]:
df_renfe_sin_duplicados_nulos.isnull().sum()

In [ ]:
df_renfe_sin_duplicados.shape

In [ ]:
df_renfe_sin_duplicados_nulos.shape

#### PREGUNTA 8

Analizamos y limpiamos el resto de variables.

In [ ]:
df_renfe_sin_duplicados_nulos.info()

In [ ]:
df_renfe_sin_duplicados_nulos_outliers = df_renfe_sin_duplicados_nulos.copy()

Analizamos variable target

In [ ]:
df_renfe_sin_duplicados_nulos_outliers["PRECIO"].hist()

In [ ]:
df_renfe_sin_duplicados_nulos_outliers[df_renfe_sin_duplicados_nulos_outliers["PRECIO"]<=0]

In [ ]:
df_renfe_sin_duplicados_nulos_outliers.drop(df_renfe_sin_duplicados_nulos_outliers[df_renfe_sin_duplicados_nulos_outliers["PRECIO"]<=0].index, axis=0, inplace=True)

In [ ]:
sns.boxplot(x = df_renfe_sin_duplicados_nulos_outliers["PRECIO"])

In [ ]:
Q1=df_renfe_sin_duplicados_nulos_outliers['PRECIO'].quantile(0.1)
Q3=df_renfe_sin_duplicados_nulos_outliers['PRECIO'].quantile(0.9)

IQR = Q3 - Q1

indexes_low = df_renfe_sin_duplicados_nulos_outliers[df_renfe_sin_duplicados_nulos_outliers['PRECIO']<(Q1-1.5*IQR)].index
indexes_high = df_renfe_sin_duplicados_nulos_outliers[df_renfe_sin_duplicados_nulos_outliers['PRECIO']>(Q3+ 1.5*IQR)].index

In [ ]:
max_precio = df_renfe_sin_duplicados_nulos_outliers[df_renfe_sin_duplicados_nulos_outliers['PRECIO']<(Q3+ 1.5*IQR)]['PRECIO'].max()
df_renfe_sin_duplicados_nulos_outliers.loc[indexes_high,'PRECIO']= max_precio

In [ ]:
sns.boxplot(x = df_renfe_sin_duplicados_nulos_outliers["PRECIO"])

Analizamos resto variable con outilers

In [ ]:
for i in df_renfe_sin_duplicados_nulos_outliers:
  print(i,df_renfe_sin_duplicados_nulos_outliers[i].dtype.kind)

In [ ]:
for i in df_renfe_sin_duplicados_nulos_outliers:
  if df_renfe_sin_duplicados_nulos_outliers[i].dtype.kind == "O":
    print(i,'\n', df_renfe_sin_duplicados_nulos_outliers[i].value_counts(), '\n')
  elif (df_renfe_sin_duplicados_nulos_outliers[i].dtype.kind=="f") or (df_renfe_sin_duplicados_nulos_outliers[i].dtype.kind=="i"):
    print(df_renfe_sin_duplicados_nulos_outliers.hist(i))

Mirando las variables categóricas, vemos que hay dos columnas en las que algunos valores aparecen muy pocas veces: TIPO_TARIFA y CLASE. Vamos a eliminarlos.

In [ ]:
# La función hace lo siguiente:
#   - como parámetros de entrada tiene un dataset, el nombre de una columna y un valor umbral
#   - crea una lista vacía
#   - crea una tabla con los value counts de la columna del dataset que le pasamos como parámetro de entrada, a la que hace un reset index
#   - se recorre las filas del data set
#   - se pregunta si el nuevo dataset que tiene los conteos de los valores es menor al umbral
#   - en caso de que sea así se agrega en una lista

def obten_lista_eliminar(dataset, columna, umbral):

  lista_borrar = []
  tabla = dataset[columna].value_counts().reset_index()
  print(tabla)

  for i in range(0,len(tabla)):
    if (tabla.iloc[i]["count"]) < umbral:
      lista_borrar.append(tabla.iloc[i][columna])

  return lista_borrar

# Con esta función podemos eliminar aquellos valores que aparecen muy pocas veces (que serían outliers) de cada variable.

In [ ]:
lista_eliminar_tarifa = obten_lista_eliminar(df_renfe_sin_duplicados_nulos_outliers,"TIPO_TARIFA",400)

In [ ]:
# Esta función tiene como parámetros de entrada un dataset, una columna y una lista.
# Elimina los valores de una columna del dataset que se encuentren en la lista.

def eliminar_lista_df(dataset, columna, lista_eliminar):
  dataset.drop(dataset[dataset[columna].isin(lista_eliminar)].index, inplace=True)

In [ ]:
eliminar_lista_df(df_renfe_sin_duplicados_nulos_outliers,"TIPO_TARIFA",lista_eliminar_tarifa)

In [ ]:
df_renfe_sin_duplicados_nulos_outliers["TIPO_TARIFA"].value_counts()

In [ ]:
lista_eliminar_clase = obten_lista_eliminar(df_renfe_sin_duplicados_nulos_outliers, "CLASE", 400)

In [ ]:
eliminar_lista_df(df_renfe_sin_duplicados_nulos_outliers,"CLASE",lista_eliminar_clase)

In [ ]:
df_renfe_sin_duplicados_nulos_outliers["CLASE"].value_counts()

In [ ]:
print("Tamaño tabla:",df_renfe_sin_duplicados_nulos_outliers.shape)

Ya hemos limpado las variables TIPO_TARIFA y CLASE.

En cuanto a las variables numéricas, vemos que DIF_INI_BUS tiene valores negativos, lo cual no tiene sentido teniendo en cuenta que esta variable es un tiempo. Vamos a eliminar estos valores.

In [ ]:
df_renfe_sin_duplicados_nulos_outliers[df_renfe_sin_duplicados_nulos_outliers["DIF_INI_BUS"]<0]

In [ ]:
df_renfe_sin_duplicados_nulos_outliers.drop(df_renfe_sin_duplicados_nulos_outliers[df_renfe_sin_duplicados_nulos_outliers["DIF_INI_BUS"]<0].index,axis=0, inplace=True)

In [ ]:
print("Tamaño tabla:",df_renfe_sin_duplicados_nulos_outliers.shape)

Ya hemos eliminado los valores negativos de DIF_INI_BUS.

Además, podemos las columnas con el detalle de la hora, el día y el mes de la consulta, puesto que la variable DIF_INI_BUS ya nos da información sobre con qué antelación se compran los billetes.

In [ ]:
for i in ("FECHA_CONSULTA_HORA", "FECHA_CONSULTA_NOMBREDIA", "FECHA_CONSULTA_DIA", "FECHA_CONSULTA_MES"):
  del(df_renfe_sin_duplicados_nulos_outliers[i])

In [ ]:
for i in ("FECHA_FIN_HORA", "FECHA_FIN_NOMBREDIA", "FECHA_FIN_DIA", "FECHA_FIN_MES"):
  del(df_renfe_sin_duplicados_nulos_outliers[i])

In [ ]:
print("Tamaño tabla:",df_renfe_sin_duplicados_nulos_outliers.shape)

In [ ]:
df_renfe_sin_duplicados_nulos_outliers.head()

#### GUARDAMOS EL DATASET EN UN FICHERO PICKLE

In [ ]:
df_renfe_sin_duplicados_nulos_outliers.to_pickle("df_renfe_clean.pkl") # Guardamos el dataset en un pickle

### PARTE 2

#### LEEMOS EL PICKLE

In [ ]:
df_renfe_clean = pd.read_pickle("df_renfe_clean.pkl") # Leemos el pickle del dataset

In [ ]:
df_renfe_clean.head()

In [ ]:
df_renfe_clean.info()

#### PREGUNTA 1

Miramos si hay variables correlacionadas.

In [ ]:
corr = df_renfe_clean.corr(numeric_only = True)

In [ ]:
corr.style.background_gradient(cmap='coolwarm')

#### PREGUNTA 2

Vamos a analizar la variable PRECIO.

In [ ]:
df_renfe_clean.hist("PRECIO")

In [ ]:
sns.boxplot(x=df_renfe_clean["PRECIO"])

Vemos que la variable PRECIO esta sesgada, vamos a crear una nueva variable LOG_PRECIO normalizada.

In [ ]:
df_renfe_clean["LOG_PRECIO"]=np.log10(df_renfe_clean["PRECIO"])

In [ ]:
df_renfe_clean.hist("LOG_PRECIO")

In [ ]:
sns.boxplot(x=df_renfe_clean["LOG_PRECIO"])

In [ ]:
df_renfe_clean.info()

#### PREGUNTA 3

Vamos a estudiar la relacion entre las variables categóricas y el precio mediante violin plots.

In [ ]:
for i in df_renfe_clean:
  print(df_renfe_clean[i].dtype.kind)

In [ ]:
for i in df_renfe_clean:
  if (df_renfe_clean[i].dtype.kind=="O") and (i!="PRECIO"):
    plt.figure(figsize=(20, 6))
    sns.violinplot(x=i, y="PRECIO", data=df_renfe_clean)

#### PREGUNTA 4

Vamos a estudiar la relación entre las variables numéricas y el precio.

In [ ]:
for i in df_renfe_clean:
  if (df_renfe_clean[i].dtype.kind in ("i","f")) and (i!="PRECIO"):
    plt.figure(figsize=(14,5))
    sns.scatterplot(x=i, y="PRECIO", data=df_renfe_clean)

#### PREGUNTA 5

Vamos a separar las variables según si son numéricas, booleanas o categóricas. Además, vamos a aplicar OHE a las variables categóricas.

In [ ]:
target= ["PRECIO","LOG_PRECIO"]

In [ ]:
def obtener_lista_variables(dataset):

    lista_numericas=[]
    lista_boolean=[]
    lista_categoricas=[]

    for i in dataset:
        if    (dataset[i].dtype.kind in ("f","i")) and len(dataset[i].unique())!= 2 and i not in target:
              lista_numericas.append(i)
        elif  (dataset[i].dtype.kind in ("f","i")) and len(dataset[i].unique())== 2 and i not in target:
              lista_boolean.append(i)
        elif  (dataset[i].dtype.kind == "O") and i not in target:
              lista_categoricas.append(i)

    return lista_numericas, lista_boolean, lista_categoricas

In [ ]:
lista_numericas, lista_boolean, lista_categoricas = obtener_lista_variables(df_renfe_clean)

In [ ]:
lista_categoricas

In [ ]:
df_renfe_prep = pd.get_dummies(data=df_renfe_clean, columns=lista_categoricas, dtype=int) # Transformamos las variables categóricas mediante OHE usando get_dummies

In [ ]:
df_renfe_prep.head()

#### PREGUNTA 6

In [ ]:
for i in lista_numericas:
  print(i)

In [ ]:
MinMax = MinMaxScaler()

In [ ]:
df_renfe_prep[lista_numericas] = MinMax.fit_transform(df_renfe_prep[lista_numericas]) # Transformamos las variables numéricas del dataset con MinMaxScaler

In [ ]:
df_renfe_prep.head()

In [ ]:
df_renfe_prep.info()

In [ ]:
corr = df_renfe_prep.corr() # correlación

In [ ]:
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
corr2 = corr.where(np.triu(np.ones(corr.shape),k=1).astype(bool)) # Nos quedamos con el triangulo superior

In [ ]:
corr2

In [ ]:
corr3 = corr2.unstack().dropna() # Borramos nulos y modificamos forma tabla

In [ ]:
corr3

In [ ]:
corr4 = corr3.sort_values() # Ordenamos los valores

In [ ]:
corr4

In [ ]:
corr4.tail(20)

In [ ]:
corr4.head(20)

Además, CIUDAD_ORIGEN_MADRID y CIUDAD_DESTINO_MADRID están inversamnte muy correlacionadas. Vamos a estudiar este caso.

In [ ]:
df_renfe_clean[df_renfe_clean["CIUDAD_DESTINO"]=="MADRID"].groupby("CIUDAD_ORIGEN")["CIUDAD_DESTINO"].agg(['count'])

In [ ]:
x = df_renfe_prep.sample(n=10)

In [ ]:
x[["CIUDAD_ORIGEN_BARCELONA","CIUDAD_DESTINO_BARCELONA","CIUDAD_ORIGEN_MADRID","CIUDAD_DESTINO_MADRID","CIUDAD_ORIGEN_VALENCIA","CIUDAD_DESTINO_VALENCIA","CIUDAD_ORIGEN_PONFERRADA","CIUDAD_DESTINO_PONFERRADA","CIUDAD_ORIGEN_SEVILLA","CIUDAD_DESTINO_SEVILLA"]].astype(int)

Se puede apreciar que **en todos los casos el origen o el destino es Madrid**. Además, **siempre que la ciudad de destino es Madrid, el origen no es Madrid**, y viceversa. **Esto es porque nuestro dataset solo contiene aquellos viajes que salen o terminan en Madrid.**

Como CIUDAD_ORIGEN_MADRID y CIUDAD_DESTINO_MADRID están muy correlacionadas, podemos eliminar una de las dos.

In [ ]:
del(df_renfe_prep["CIUDAD_ORIGEN_MADRID"])

#### VAMOS A GUARDAR EL DATASET EN UN FICHERO PICKLE

In [ ]:
df_renfe_prep.to_pickle("dataset_preprocesado.pkl")

### ¡VAMOS A HACER UN MODELO DE ML!

In [ ]:
from sklearn.model_selection import train_test_split # Librería para el separar los datasets de train y test

In [ ]:
df_renfe_final = pd.read_pickle("dataset_preprocesado.pkl") # Leemos el pickle del dataset

In [ ]:
del(df_renfe_final["LOG_PRECIO"]) # Borramos el LOG_PRECIO, ya que en este caso haremos la predicción sobre PRECIO por simplicidad

In [ ]:
df_renfe_final.head()

In [ ]:
X = df_renfe_final.drop(['PRECIO'],axis=1) # VARIABLES INDEPENDIENTES
y = df_renfe_final['PRECIO'] # VARIABLE DEPENDIENTE (A PREDECIR)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=23) # Separamos los datasets de train y test

In [ ]:
from sklearn.linear_model import LinearRegression # Librería para generar un modelo de regresión lineal

In [ ]:
lin_reg = LinearRegression() # Modelo de regresión lineal

In [ ]:
lin_reg.fit(X_train,y_train) # Entrenamos el modelo con los datasets de train

In [ ]:
y_pred = lin_reg.predict(X_test) # Predecimos con el modelo usando los datasets de test

In [ ]:
df_resultados = pd.DataFrame({'Actual':y_test, 'Predicted':y_pred}) # Creamos un dataset con el precio predecido y el precio real del dataset de test

In [ ]:
df_resultados.head(10)

In [ ]:
df_resultados_ext = [df_resultados,X_test]
df_resultados_ext2 = pd.concat(df_resultados_ext,axis=1)

In [ ]:
df_resultados_ext2.head(10)

In [ ]:
df_resultados.hist("Actual",range=[0,200])

In [ ]:
df_resultados.hist("Predicted",range=[0,200])

In [ ]:
from sklearn import metrics # Importamos métricas para validar el modelo

In [ ]:
print ("MAE:", metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
np.mean(df_renfe_final["PRECIO"])

In [ ]:
df_resultados["dif"]=df_resultados["Predicted"]-df_resultados["Actual"]

In [ ]:
df_resultados.sort_values(by="dif")

In [ ]:
df_resultados.hist("dif",range=[-60,60])